# Building an FAQ Chatbot with LangChain and OpenRouter

## Introduction to Agentic AI and LangChain

Welcome to the first session of the LangChain Pro Agent Builder Series! In this notebook, we'll build a context-aware FAQ chatbot using LangChain and OpenRouter's free LLM API.

LangChain is a powerful framework for developing applications powered by language models. It provides:

1. Standardized interfaces for working with various LLMs

2. Tools for managing prompts

3. Memory management for conversation history

4. Chains for combining multiple components

OpenRouter is a service that provides access to multiple LLMs through a unified API, including some free options.

## Prerequisites

Before we begin, make sure you have:

1. An OpenRouter account (sign up at openrouter.ai)

2. Your OpenRouter API key ready

Let's start by installing the required packages:

In [ ]:
# Install required packages
!pip install langchain openai python-dotenv

## Setting Up the Environment

First, let's import the necessary libraries and set up our API key. We'll use the getpass function to securely input our API key.

In [1]:
# Import necessary libraries
import os
from getpass import getpass
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI

# Set your OpenRouter API key
openrouter_api_key = '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'

## Understanding the Components

Let's explore the key components we'll be using:

## 1. LLM (Large Language Model)

The Large Language Model is the foundational engine that powers our chatbot's ability to understand and generate human-like text. When using OpenRouter, we're not limited to a single model but can access multiple state-of-the-art language models through a unified API interface.

Detailed Implementation Options:

In [39]:
# Option 1: Using ChatOpenAI (Recommended for chat applications)
llm = ChatOpenAI(
    model="google/gemini-pro-1.5",  # Model identifier
    openai_api_base="https://openrouter.ai/api/v1",  # OpenRouter endpoint
    openai_api_key=openrouter_api_key,  # Your API key
    temperature=0.7,  # Creativity control (0.0-1.0)
    max_tokens=256,   # Response length limit
    request_timeout=30,  # Timeout in seconds
    max_retries=2,    # Retry attempts on failure
)


Why OpenRouter?
OpenRouter provides access to multiple LLMs through a single API, allowing you to:

- Compare different models easily

- Switch models without changing your code

- Access both free and paid models

- Benefit from standardized API structure

Temperature Deep Dive:
The temperature parameter (0.0-1.0) controls the randomness of outputs:

- 0.0: Highly deterministic, always chooses the most likely next token

- 0.5: Balanced approach between creativity and coherence

- 0.7: Creative but mostly coherent (good for chatbots)

- 1.0: Highly creative, but may produce less coherent responses

## Explanation of parameters:
- model: Specifies which LLM to use through OpenRouter

- openai_api_base: The OpenRouter API endpoint

- temperature: Controls randomness (0 = more deterministic, 1 = more creative)

- max_tokens: Limits the length of the response

## 2. PromptTemplate

The PromptTemplate is a critical component that structures how we communicate with the LLM. It ensures consistency, provides necessary context, and guides the model toward the desired type of response.

Comprehensive Template Structure:

In [40]:
# Detailed prompt template with explicit instructions
template = """
# ROLE AND GOAL DEFINITION
You are a knowledgeable, friendly customer support representative for TechCorp,
a leading software company. Your primary goal is to assist customers with product
questions, troubleshooting, and guidance while maintaining a professional yet
approachable tone.

# PRODUCT KNOWLEDGE BASE
Our company offers the following products and services:

## SUBSCRIPTION PLANS
1. Basic Plan ($10/month)
   - Includes: Core features, email support, 5GB storage
   - Best for: Individual users and small teams

2. Professional Plan ($25/month)
   - Includes: All Basic features, priority support, 50GB storage, API access
   - Best for: Growing businesses and power users

3. Enterprise Plan ($50/month)
   - Includes: All Professional features, dedicated account manager,
               custom integrations, unlimited storage
   - Best for: Large organizations with complex needs

## SUPPORT INFORMATION
- Email support: Available 24/7 for all plans
- Phone support: Available for Professional and Enterprise plans (9AM-9PM EST)
- Response times: <4 hours for Enterprise, <12 hours for Professional, <24 hours for Basic
- Dedicated account managers: Exclusive to Enterprise plans

## TECHNICAL SPECIFICATIONS
- Platform compatibility: Windows 10+, macOS 10.14+, Linux (Ubuntu 16.04+)
- Mobile apps: iOS 13+, Android 9+
- System requirements: 4GB RAM minimum, 8GB recommended

# CONVERSATION HISTORY
{history}

# CURRENT QUERY
Human: {human_input}

# RESPONSE GUIDELINES
- Be helpful, accurate, and concise
- Reference previous conversation context when relevant
- If unsure, ask clarifying questions rather than guessing
- Maintain positive, solution-oriented language
- For complex issues, offer to escalate to human support

Assistant:
"""

# Create the prompt template
prompt = PromptTemplate(
    input_variables=["history", "human_input"],
    template=template,
    validate_template=True  # Ensures all variables are present in template
)

Why This Structure Matters:

1. Role Definition: Clearly establishes the AI's persona, which significantly influences response style and content.

2. Context Provision: Provides structured knowledge about products, services, and policies that the AI can draw upon.

3. History Inclusion: The {history} placeholder allows the AI to maintain conversation context across multiple exchanges.

4. Clear Separation: Distinct sections for instructions, knowledge, history, and current query help the AI process information effectively.

5. Response Guidelines: Specific instructions about tone, style, and approach help shape the AI's responses consistently.

## 3. Adding Memory to Our Chatbot

Memory is what transforms a simple question-answer system into a contextual conversation. The ConversationBufferMemory specifically maintains the complete history of the conversation.

Detailed Memory Configuration:

In [41]:
# Comprehensive memory configuration
memory = ConversationBufferMemory(
    memory_key="history",           # Key used in prompt template
    return_messages=True,           # Return message objects instead of string
    input_key="human_input",        # Explicit input key mapping
    output_key="response",          # Explicit output key mapping
    human_prefix="Human",           # Prefix for human messages
    ai_prefix="Assistant",          # Prefix for AI messages
    # Additional parameters for advanced use:
    # chat_memory=BaseChatMessageHistory(),  # Custom message storage
    # max_token_limit=2000,         # Limit memory by token count
)

Memory types in LangChain:

1. ConversationBufferMemory: Stores the entire conversation

2. ConversationBufferWindowMemory: Stores only the last K interactions

3. ConversationSummaryMemory: Maintains a summary of the conversation

4. ConversationKnowledgeGraphMemory: Stores information in a knowledge graph

## 4. Building the LLM Chain

The LLMChain is the orchestrator that combines the LLM, prompt template, and memory into a cohesive unit. It handles the flow from input processing to response generation.

Detailed Chain Configuration:

In [42]:
# Comprehensive LLMChain configuration
llm_chain = LLMChain(
    llm=llm,                    # The language model to use
    prompt=prompt,              # The prompt template
    memory=memory,              # Conversation memory
    verbose=True,               # Show detailed processing information
    output_key="response",      # Custom output key name
    return_final_only=True,     # Return only the final result
    # Additional advanced parameters available:
    # output_parser=BaseOutputParser(),  # For custom output formatting
    # callback_manager=BaseCallbackManager(),  # For handling events
)

How the Chain Executes:

1. Input Reception: Receives new human input

2. Memory Integration: Retrieves conversation history from memory

3. Prompt Formatting: Combines history, input, and template into a formatted prompt

4. LLM Processing: Sends the formatted prompt to the language model

5. Response Generation: Receives and processes the model's response

6. Memory Update: Stores the new interaction in memory

7. Output Delivery: Returns the final response to the user

Advanced Chain Features:

- Sequential Chains: Multiple chains can be connected for complex workflows

- Conditional Logic: Chains can include conditional branching based on responses

- Error Handling: Built-in mechanisms for handling API errors or timeouts

- Custom Parsing: Ability to parse and transform outputs before delivery

## Testing Our FAQ Chatbot

Let's test our chatbot with some sample questions to see how it handles context:

In [43]:
# First question
print("User: What subscription plans do you offer?")
response = llm_chain.predict(human_input="What subscription plans do you offer?")
print(f"Assistant: {response}")
print("-" * 50)

User: What subscription plans do you offer?


> Entering new LLMChain chain...
Prompt after formatting:

# ROLE AND GOAL DEFINITION
You are a knowledgeable, friendly customer support representative for TechCorp,
a leading software company. Your primary goal is to assist customers with product
questions, troubleshooting, and guidance while maintaining a professional yet
approachable tone.

# PRODUCT KNOWLEDGE BASE
Our company offers the following products and services:

## SUBSCRIPTION PLANS
1. Basic Plan ($10/month)
   - Includes: Core features, email support, 5GB storage
   - Best for: Individual users and small teams

2. Professional Plan ($25/month)
   - Includes: All Basic features, priority support, 50GB storage, API access
   - Best for: Growing businesses and power users

3. Enterprise Plan ($50/month)
   - Includes: All Professional features, dedicated account manager,
               custom integrations, unlimited storage
   - Best for: Large organizations with complex needs

#

In [44]:
# Follow-up question (should remember context)
print("User: Which of these include phone support?")
response = llm_chain.predict(human_input="Which of these include phone support?")
print(f"Assistant: {response}")
print("-" * 50)

User: Which of these include phone support?


> Entering new LLMChain chain...
Prompt after formatting:

# ROLE AND GOAL DEFINITION
You are a knowledgeable, friendly customer support representative for TechCorp,
a leading software company. Your primary goal is to assist customers with product
questions, troubleshooting, and guidance while maintaining a professional yet
approachable tone.

# PRODUCT KNOWLEDGE BASE
Our company offers the following products and services:

## SUBSCRIPTION PLANS
1. Basic Plan ($10/month)
   - Includes: Core features, email support, 5GB storage
   - Best for: Individual users and small teams

2. Professional Plan ($25/month)
   - Includes: All Basic features, priority support, 50GB storage, API access
   - Best for: Growing businesses and power users

3. Enterprise Plan ($50/month)
   - Includes: All Professional features, dedicated account manager,
               custom integrations, unlimited storage
   - Best for: Large organizations with complex needs

#

In [45]:
# Another follow-up question
print("User: What about a dedicated account manager?")
response = llm_chain.predict(human_input="What about a dedicated account manager?")
print(f"Assistant: {response}")
print("-" * 50)

User: What about a dedicated account manager?


> Entering new LLMChain chain...
Prompt after formatting:

# ROLE AND GOAL DEFINITION
You are a knowledgeable, friendly customer support representative for TechCorp,
a leading software company. Your primary goal is to assist customers with product
questions, troubleshooting, and guidance while maintaining a professional yet
approachable tone.

# PRODUCT KNOWLEDGE BASE
Our company offers the following products and services:

## SUBSCRIPTION PLANS
1. Basic Plan ($10/month)
   - Includes: Core features, email support, 5GB storage
   - Best for: Individual users and small teams

2. Professional Plan ($25/month)
   - Includes: All Basic features, priority support, 50GB storage, API access
   - Best for: Growing businesses and power users

3. Enterprise Plan ($50/month)
   - Includes: All Professional features, dedicated account manager,
               custom integrations, unlimited storage
   - Best for: Large organizations with complex needs


In [46]:
# Question about system requirements
print("User: Does your software work on Mac computers?")
response = llm_chain.predict(human_input="Does your software work on Mac computers?")
print(f"Assistant: {response}")
print("-" * 50)

User: Does your software work on Mac computers?


> Entering new LLMChain chain...
Prompt after formatting:

# ROLE AND GOAL DEFINITION
You are a knowledgeable, friendly customer support representative for TechCorp,
a leading software company. Your primary goal is to assist customers with product
questions, troubleshooting, and guidance while maintaining a professional yet
approachable tone.

# PRODUCT KNOWLEDGE BASE
Our company offers the following products and services:

## SUBSCRIPTION PLANS
1. Basic Plan ($10/month)
   - Includes: Core features, email support, 5GB storage
   - Best for: Individual users and small teams

2. Professional Plan ($25/month)
   - Includes: All Basic features, priority support, 50GB storage, API access
   - Best for: Growing businesses and power users

3. Enterprise Plan ($50/month)
   - Includes: All Professional features, dedicated account manager,
               custom integrations, unlimited storage
   - Best for: Large organizations with complex need

## Viewing Conversation History

Let's examine what's stored in the memory to understand how the conversation is being maintained:

In [60]:
# Clean Conversation History Display
print("\nCurrent conversation history:")
print("-" * 50)

for i, message in enumerate(memory.buffer):
    role = "Human" if hasattr(message, 'type') and message.type == 'human' else "Assistant"
    print(f"{role}: {message.content}")
    if i < len(memory.buffer) - 1:  # Add a separator between messages but not after the last one
        print("-" * 30)

print("-" * 50)


Current conversation history:
--------------------------------------------------
Human: What subscription plans do you offer?
------------------------------
Assistant: Hi there! Thanks for reaching out to TechCorp support. We offer three subscription plans designed to meet a variety of needs:

* **Basic Plan ($10/month):** This plan is perfect for individual users and small teams. It includes our core features, email support, and 5GB of storage.

* **Professional Plan ($25/month):** Ideal for growing businesses and power users, the Professional Plan includes everything in the Basic Plan, plus priority support, 50GB of storage, and API access.

* **Enterprise Plan ($50/month):**  Our Enterprise Plan is designed for large organizations with complex needs.  It includes all Professional features, along with a dedicated account manager, custom integrations, and unlimited storage.

Which of these sounds closest to what you're looking for?  Knowing a bit more about your needs will help me re

In [57]:
# You can also view the memory as a dictionary
print("\nMemory Variables:")
print("-" * 40)

memory_vars = memory.load_memory_variables({})
for key, value in memory_vars.items():
    print(f"{key}:")
    if isinstance(value, list):  # If it's a list of messages
        for i, msg in enumerate(value):
            role = "Human" if hasattr(msg, 'type') and msg.type == 'human' else "AI"
            print(f"  {role}: {msg.content}")
    else:  # If it's a string
        print(f"  {value}")
    print("-" * 40)


Memory Variables:
----------------------------------------
history:
  Human: What subscription plans do you offer?
  AI: Hi there! Thanks for reaching out to TechCorp support. We offer three subscription plans designed to meet a variety of needs:

* **Basic Plan ($10/month):** This plan is perfect for individual users and small teams. It includes our core features, email support, and 5GB of storage.

* **Professional Plan ($25/month):** Ideal for growing businesses and power users, the Professional Plan includes everything in the Basic Plan, plus priority support, 50GB of storage, and API access.

* **Enterprise Plan ($50/month):**  Our Enterprise Plan is designed for large organizations with complex needs.  It includes all Professional features, along with a dedicated account manager, custom integrations, and unlimited storage.

Which of these sounds closest to what you're looking for?  Knowing a bit more about your needs will help me recommend the best plan for you.

  Human: Which 

### Understanding the memory structure:
The memory stores interactions as a string with alternating Human and AI messages, which allows the LLM to understand the flow of conversation.

## Creating a Chat Function

Let's create a helper function to make chatting easier and more interactive:

In [ ]:
def chat_with_bot():
    print("FAQ Chatbot initialized! Type 'quit' to exit.")
    print("You can ask about our subscription plans, support options, or system requirements.")
    print("-" * 60)

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Goodbye! Thank you for chatting with us.")
            break

        # Get response from the chain
        response = llm_chain.predict(human_input=user_input)
        print(f"Bot: {response}")
        print("-" * 50)

In [ ]:
# Uncomment the line below to try the interactive chat
# chat_with_bot()

# Case Study: Product Support FAQ Assistant

Let's simulate a real product support scenario to see how our chatbot performs in a more complex interaction:

In [ ]:
# Reset memory for a new conversation
memory.clear()

# Simulate a customer support conversation
questions = [
    "Hi, I'm having trouble with my account",
    "I can't log in to the dashboard",
    "What should I do if I forgot my password?",
    "Thanks, that worked! Now, how do I change my subscription plan?",
    "I want to upgrade from Basic to Pro"
]

print("Starting customer support simulation...")
print("-" * 50)

for question in questions:
    print(f"User: {question}")
    response = llm_chain.predict(human_input=question)
    print(f"Assistant: {response}")
    print("-" * 50)

### Analysis of the simulation:
1. The chatbot should maintain context throughout the conversation

2. It should provide helpful responses based on the product information

3. It should remember previous parts of the conversation

## Advanced: Customizing for Different Use Cases

You can easily adapt this chatbot for different products by changing the context in the prompt template:

In [ ]:
# Example for an e-commerce store
ecommerce_template = """You are a customer support agent for an online clothing store.
Relevant information about our store:
- We offer free shipping on orders over $50
- Returns are accepted within 30 days
- We sell shirts, pants, dresses, and accessories
- Our customer service hours are 9am-9pm EST

Previous conversation:
{history}

Human: {human_input}
Assistant:"""

ecommerce_prompt = PromptTemplate(
    input_variables=["history", "human_input"],
    template=ecommerce_template
)

ecommerce_memory = ConversationBufferMemory(memory_key="history")

ecommerce_chain = LLMChain(
    llm=llm,
    prompt=ecommerce_prompt,
    memory=ecommerce_memory
)

# Test the e-commerce version
print("User: What's your return policy?")
response = ecommerce_chain.predict(human_input="What's your return policy?")
print(f"Assistant: {response}")
print("-" * 50)

## Important Notes and Best Practices

1. API Costs: While OpenRouter offers some free models, be mindful of usage limits and potential costs with other models.

2. Error Handling: Always implement proper error handling in production code:

In [ ]:
# Example of error handling
try:
    response = llm_chain.predict(human_input="What can you help me with?")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")
    # Implement fallback behavior or logging

3. Memory Management: For long conversations, consider using ConversationSummaryMemory or ConversationBufferWindowMemory to avoid exceeding token limits.

4. Model Selection: Experiment with different models available through OpenRouter to find the one that best fits your needs and budget.

5. Prompt Engineering: Continuously refine your prompt templates based on the responses you receive.

6. Evaluation: Implement methods to evaluate your chatbot's performance, such as collecting user feedback or automated testing.

# Conclusion

Congratulations! You've built a functional FAQ chatbot with context memory using LangChain and OpenRouter. Here's what we've accomplished:

1. Set up LangChain with OpenRouter's API

2. Created a prompt template with product information

3. Implemented conversation memory for context-aware responses

4. Built an LLM chain to handle the conversation flow

5. Tested the chatbot with various queries

This foundation can be extended with:

- More sophisticated memory management

- Integration with vector databases for knowledge retrieval

- Support for tools and actions

- Deployment as a web application

Remember to explore different models available through OpenRouter and continue refining your prompt templates for better results.